In [2]:
import pandas as pd
import collections
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
import re
import collections
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option('display.max_column',4000)
pd.set_option('display.max_row',4000)


# 1. Reading Training and Test data into dataframe

In [3]:
tr=pd.read_csv("P3_training_data.csv",header=None)
te=pd.read_csv("P3_test_data.csv",header=None)
print(te)
tr.columns=['text']
tr=tr['text'].str.split(n=1,expand=True)
tr.columns=['class','text']
te.columns=['text']
te['class']='none'
print(tr)
print(te)

                                                     0
0     those of you who frequently read my reviews a...
1     this is a stagy film adapted from roger rueff...
2     the coen brothers are back again   this time ...
3     contact   pg   there s a moment late in rober...
4     satirical films usually fall into one of two ...
5     after sixteen years francis ford copolla has ...
6     bruce willis is a type casted actor    in die...
7     there must be some unwritten rule that states...
8     did claus von bulow try to kill his wife sunn...
9     driving miss daisy takes its sweet time to te...
10    when people are talking about good old times ...
11    note   ordinarily   moviereviews   org will n...
12    seen april 16   1999 at 10 p   m   at crossga...
13    mars attacks !     1996     c   jack nicholso...
14    when i was asked to see this movie with a fri...
15    you d think it would be easy to spoof star tr...
16    in my review of there s something about mary ...
17    show

# 2. DEFINING FUNCTIONS FOR STOPWORD REMOVAL,STEMMIZATION AND           CLEANING PUNCTUATIONS


In [4]:
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
stop = set(stopwords.words('english'))
print(stop)
sno = nltk.stem.SnowballStemmer('english')
def preprocessing(x):
    final_string=[]
    s=''
    for text in x['text'].values:
        filtered_sentence=[]
        for w in text.split():
            for cleaned_words in cleanpunc(w).split():
                if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                    if(cleaned_words.lower() not in stop):
                        s=(sno.stem(cleaned_words.lower())).encode('utf8')
                        filtered_sentence.append(s)
               
                    else:
                        continue
                else:
                    continue 
        str1 = b" ".join(filtered_sentence)
        str1=str1.decode('utf8')
        final_string.append(str1)

    x['text']=final_string


{'over', 'too', 'the', 'off', "you'll", "should've", "doesn't", 'but', 'other', "she's", 've', 'from', 'we', 'on', "needn't", 'will', 'up', 'below', "weren't", 'his', 'each', 'mightn', 'he', 'all', 'out', "don't", 'who', 'as', 'needn', "you'd", 'hasn', 'own', 'these', "hasn't", 'myself', 'which', 'about', 'until', "it's", 'did', 'or', 'then', 'of', 'once', "you've", 'a', 'what', 'such', 'himself', 'those', "didn't", 'herself', 'and', 'here', 'do', 'between', 'you', 'them', 'couldn', 'being', 'be', 'i', "aren't", 'whom', "couldn't", 'haven', 'didn', 'doesn', 'y', 'hadn', 'ours', 'is', 'itself', 'are', 'your', 'll', 'does', 'after', 'how', 't', "shan't", 'now', 'had', 'shouldn', 'our', 'into', 'having', 'in', 'ain', "haven't", 'again', 'isn', 'both', 'her', 'no', 'it', 'through', 'while', 'she', "you're", 'under', "shouldn't", 'yourself', 'shan', 'some', 'am', 'because', 'when', 'don', 'can', 'hers', "that'll", 'further', 'not', 'very', 'ma', 'most', 'just', 'during', 'so', 'mustn', 'sho

# 3. Apply Preprocessing to both training and test data

In [5]:
preprocessing(tr)
preprocessing(te)
print(tr)


     class                                               text
0      Pos  film adapt comic book plenti success whether s...
1      Pos  everi movi come along suspect studio everi ind...
2      Pos  got mail work alot better deserv order make fi...
3      Pos  jaw rare film grab attent show singl imag scre...
4      Pos  moviemak lot like general manag nfl team post ...
5      Pos  june self taught idealist yet pragmat young ma...
6      Pos  appar director toni kay major battl new line r...
7      Pos  one colleagu surpris told will see betsi wed s...
8      Pos  bloodi clash independ lumumba refus pander bel...
9      Pos  american action film slowli drown death sea as...
10     Pos  watch rat race last week notic cheek sore real...
11     Pos  notic someth late never thought pseudo substan...
12     Pos  synopsi bobbi garfield yelchin live small town...
13     Pos  synopsi movi steven spielberg one today finest...
14     Pos  polic negoti person entir unenvi job go ground...
15     P

# 4. Seperating training data to positive and negative class

In [6]:
tr_pos=tr[tr['class']=='Pos']
tr_neg=tr[tr['class']=='Neg']

In [7]:
tr_pos_count=tr_pos.shape[0]
tr_neg_count=tr_neg.shape[0]
prior_pos=tr_pos_count/(tr_pos_count+tr_neg_count)
prior_neg=1-prior_pos

# 5. Creating multinomial BOW representations for both positive and negative class words

In [8]:
pos_voc=dict(collections.Counter([y for x in tr_pos.text.values.flatten() for y in x.split()]))
neg_voc=dict(collections.Counter([y for x in tr_neg.text.values.flatten() for y in x.split()]))
print(neg_voc)

{'plot': 781, 'two': 723, 'teen': 100, 'coupl': 205, 'church': 18, 'parti': 65, 'drink': 45, 'drive': 92, 'get': 1383, 'accid': 37, 'one': 2265, 'guy': 431, 'die': 189, 'girlfriend': 105, 'continu': 93, 'see': 793, 'life': 424, 'nightmar': 34, 'deal': 118, 'watch': 471, 'movi': 3000, 'sorta': 3, 'find': 415, 'critiqu': 27, 'mind': 205, 'fuck': 17, 'generat': 102, 'touch': 74, 'cool': 106, 'idea': 244, 'present': 121, 'bad': 884, 'packag': 15, 'make': 1216, 'review': 198, 'even': 1111, 'harder': 11, 'write': 164, 'sinc': 311, 'general': 105, 'applaud': 4, 'film': 4046, 'attempt': 277, 'break': 117, 'mold': 11, 'mess': 108, 'head': 233, 'lost': 185, 'highway': 15, 'memento': 3, 'good': 978, 'way': 683, 'type': 107, 'folk': 55, 'snag': 2, 'correct': 21, 'seem': 788, 'taken': 85, 'pretti': 236, 'neat': 20, 'concept': 49, 'execut': 70, 'terribl': 125, 'problem': 319, 'well': 676, 'main': 188, 'simpli': 173, 'jumbl': 9, 'start': 316, 'normal': 41, 'downshift': 2, 'fantasi': 48, 'world': 303,

In [9]:
voc=set(list(pos_voc.keys())+list(neg_voc.keys()))
voc_size=np.size(list(voc))
print(voc_size)

22550


In [10]:
pos_voc_count=0
for value in pos_voc.values():
    pos_voc_count=pos_voc_count+value
neg_voc_count=0
for value in neg_voc.values():
    neg_voc_count=neg_voc_count+value


# 6. Implementing Bayes Classifier for predicted the class for test data

In [11]:
def pos_prob(x):
    if x in pos_voc.keys():
        x_count=pos_voc[x]+1
    elif (x in neg_voc.keys()):
        x_count=1
    else:
        return 0
    
    return np.log((x_count/(pos_voc_count+1*voc_size)))
def neg_prob(x):
    if x in neg_voc.keys():
        x_count=neg_voc[x]+1
    elif (x in pos_voc.keys()):
        x_count=1
    else:
        return 0
    
    return np.log((x_count/(neg_voc_count+1*voc_size)))

In [12]:
output=[]

for text in te['text']:
    
    l=text.split()
    pos_prob_total=0
    neg_prob_total=0
    for i in l:
        pos_prob_total=pos_prob_total+pos_prob(i)
        neg_prob_total=neg_prob_total+neg_prob(i)
    if(pos_prob_total>neg_prob_total):
        output=output+[1]
    else:
        output=output+[0]
print(len(output))
print(output)

400
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,